In [12]:
import cv2
import numpy as np
from PIL import Image
import os

# Define your directory paths
image_dir = '/Documents/DL_LABS/PROJECT/data/val/AML'  
# mask_dir = '/Documents/DL_LABS/PROJECT/masks/train_seg_masks_resized/CML'  
mask_dir = '/Documents/DL_LABS/PROJECT/masks/val_norm_mask_resized/AML'   
output_dir = 'Documents/DL_LABS/PROJECT/patched_data/val/AML' 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Desired size for resizing images and masks
resize_dims = (448, 448)

# Normalize filename by stripping and replacing unwanted characters
def normalize_filename(filename):
    """Normalize filenames by removing extra spaces and standardizing format."""
    filename = filename.strip()
    parts = filename.split()
    normalized_parts = []
    for part in parts:
        if '-' in part:
            part = '-'.join(part.split('-'))
        normalized_parts.append(part)
    return ' '.join(normalized_parts)

In [13]:
# Iterate over the images in the image directory
for filename in os.listdir(image_dir):
    filename = normalize_filename(filename)
    if any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.tif', '.tiff']):
        base_name = os.path.splitext(filename)[0]
        mask_filename = f"{base_name}.png" if base_name.endswith('_mask') else f"{base_name}_mask.png"
        image_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, mask_filename)

        # Load the original image and mask
        original_image = cv2.imread(image_path)
        mask_image = cv2.imread(mask_path, 0)

        if original_image is None:
            print(f"Failed to load image {filename}.")
            continue
        if mask_image is None:
            print(f"Failed to load mask for {filename}. Check file: {mask_path}")
            continue

        # Resize the original image and mask to 448x448
        original_image_resized = cv2.resize(original_image, (448, 448))

        # Optional: Adjust mask processing here, e.g., apply threshold
        _, mask_image = cv2.threshold(mask_image, 127, 255, cv2.THRESH_BINARY)

        # Find contours in the mask image
        contours, _ = cv2.findContours(mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cell_count = 0  # Initialize cell count
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 100:  # Ignore contours with area less than 100 pixels
                continue

            x, y, w, h = cv2.boundingRect(contour)
            cropped_img = original_image_resized[y:y+h, x:x+w]

            # Resize the cell to 232x232 without padding
            resized_img = cv2.resize(cropped_img, (100, 100))
            pil_img = Image.fromarray(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))

            # Save each cell with a unique filename
            cell_count += 1
            cell_filename = f"{os.path.splitext(filename)[0]}_{cell_count}.png"
            cell_output_path = os.path.join(output_dir, cell_filename)
            pil_img.save(cell_output_path)
            print(f"Saved cell {cell_filename}")

        # Print a message if no valid contours were found
        if not contours:
            print(f"No valid contours found for {filename}, saving resized original image.")

print("All images processed.")

Saved cell Image_3820_1.png
Saved cell Image_3820_2.png
Saved cell 44-23-063-1500 - 10_1.png
Saved cell 44-23-063-1500 - 10_2.png
Saved cell 44-23-063-1500 - 10_3.png
Saved cell 44-23-063-1500 - 10_4.png
Saved cell 44-23-063-1500 - 10_5.png
Saved cell 44-23-063-1500 - 10_6.png
Saved cell 44-23-063-1500 - 10_7.png
Saved cell 44-23-063-1500 - 10_8.png
Saved cell Image_3827_1.png
Saved cell Image_3855_1.png
Saved cell Image_3829_1.png
Saved cell 44-23-117-1842 - 9_1.png
Saved cell 44-23-117-1842 - 9_2.png
Saved cell 44-23-117-1842 - 9_3.png
Saved cell 44-23-117-1842 - 9_4.png
Saved cell 44-23-117-1842 - 9_5.png
Saved cell 44-23-117-1842 - 9_6.png
Saved cell 44-23-117-1842 - 9_7.png
Saved cell 44-23-117-1842 - 9_8.png
Saved cell 44-23-117-1842 - 9_9.png
Saved cell 44220980309A_40_1.png
Saved cell Image_3852_1.png
Saved cell Image_3943_1.png
Saved cell Image_3943_2.png
Saved cell Image_3938_1.png
Saved cell Image_3860_1.png
Saved cell Image_3860_2.png
Saved cell Image_3944_1.png
Saved cell 